### <b>The first big project - Professionally You!</b> (Career Alter Ego)

<b> And, Tool use. </b>

In [ ]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import json, os, requests, gradio as gr

In [ ]:
# The usual start

load_dotenv(override=True)

openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### <b>Pushover setup</b>

Pushover is a nifty tool for sending Push Notifications to your phone.

Make sure to include PUSHOVER_USER and PUSHOVER_TOKEN in `.env` file and run `load_dotenv(override=True)` 
after saving, to set the environment variables

In [ ]:

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user is set and starts with: {pushover_user[0]}")
else:
    print("Pushover user is not set")

if pushover_token:
    print(f"Pushover token is set and starts with: {pushover_token[0]}")
else:
    print("Pushover token is not set")

In [ ]:
# Function to send a push notification

def send_push_notification(message):
    payload = {
        "token": pushover_token,
        "user": pushover_user,
        "message": message
    }
    requests.post(pushover_url, data=payload)
    
    print(f"Push notification sent: {message}")

In [ ]:
send_push_notification("Hello!")

### <b>Tools setup</b>

Now, we will define two functions intended for different purposes, so that they can be used as tools by the LLM

To achieve this, we need to have tool definitions in JSON format for the LLM to understand the details. Once defined, we will package them into a tools list

In [ ]:
# Function to record details of user that is interested in being in touch

def record_user_details(email, name="not provided", notes="not provided"):
    """Records details of a user who is interested in being in touch."""
    if not email:
        return "Error: Email is required to record user details."

    send_push_notification(f"Recording interest from {name} with email {email} and notes {notes}")

    # Return a simple string to confirm success
    return f"Successfully logged details for {name} ({email})."    

# Function to record unanswered questions

def record_unanswered_question(question):
    """Records a user's question that the model could not answer."""

    if not question:
        return "Error: No question was provided to record."

    send_push_notification(f"Recording this question that I couldn't answer: {question}")

    # Return a simple string to confirm success
    return f"Successfully logged question: {question}"


In [ ]:
# Tool Definitions in JSON format
#
# Using UPPER_SNAKE_CASE, as these are module-level constants.
# The suffix "_TOOL" clearly states their purpose.

RECORD_USER_DETAILS_TOOL = {
    "name": "record_user_details",
    "description": "Records details of a user who is interested in being in touch.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The user's email address.",
            },
            "name": {
                "type": "string",
                "description": "The user's name.",
            },
            "notes": {
                "type": "string",
                "description": "Any additional notes the user provided.",
            },
        },
        "required": ["email"],
        "additionalProperties": False 
    }
}

RECORD_UNANSWERED_QUESTION_TOOL = {
    "name": "record_unanswered_question",
    "description": "Always use this tool to record any question that the user asked but couldn't be answered.",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            }
        },
        "required": ["question"],
        "additionalProperties": False 
    }
}

In [ ]:
# Assemble the Final Tools List

tools = [
    {"type": "function", "function": RECORD_USER_DETAILS_TOOL},
    {"type": "function", "function": RECORD_UNANSWERED_QUESTION_TOOL}
]

In [ ]:
tools

In [ ]:
# This function can take a list of tool calls from the LLM, and run them. This is the IF statement!!

def handle_tool_calls_OLD(tool_calls):
    """
    Takes a list of tool_calls from an OpenAI response, executes them,
    and returns a list of "tool" messages.
    """
    tool_output = []

    function_map = {
        'record_user_details': record_user_details,
        'record_unanswered_question': record_unanswered_question
    }

    for tool_call in tool_calls:

        function_name = tool_call.function.name
        # 1. Parse the JSON string of arguments
        function_args = json.loads(tool_call.function.arguments)
        
        print(f"Tool called: {function_name}", flush=True)

        # Get the function to call based on the function name
        # THE BIG IF STATEMENT!!!
        """
        if function_name == "record_user_details":
            result = record_user_details(**function_args)
        elif function_name == "record_unknown_question":
            result = record_unanswered_question(**function_args)
        """

        function_to_call = function_map[function_name]

        # 2. Call the function using ** to unpack the dictionary as arguments
        function_response = function_to_call(**function_args)

        tool_output.append({"role": "tool","tool_call_id": tool_call.id,"content": json.dumps(function_response)})

    return tool_output
        


Instead of using the If statement or the function map dictionary, we can use Python's <b>global()</b> function
to find the function to call, based on the incoming tool name

<b>global()</b> is a built-in Python function that returns a dictionary of all global names (variables, functions, classes, etc.) that exist in the current module.

In [ ]:
# print the function name from globals
print(globals()["record_unanswered_question"])

# execute the function. This sends a push notification
globals()["record_unanswered_question"]("This is a really tough question")

In [ ]:
# This is a more elegant way that avoids the IF statement/function.

def handle_tool_calls(tool_calls):
    """
    Takes a list of tool_calls from an OpenAI response, executes them,
    and returns a list of "tool" messages.
    """
    tool_output = []

    for tool_call in tool_calls:
        # Get the function name and the function to call
        function_name = tool_call.function.name
        function_to_call = globals().get(function_name)

        if not function_to_call:
            # Handle cases where the model tries to call a function that doesn't exist
            function_response = f"Error: Function '{function_name}' not found."
        else:
            try:
                # 1. Parse the JSON string of arguments
                function_args = json.loads(tool_call.function.arguments)
                # 2. Call the function using ** to unpack the dictionary as arguments
                print(f"Tool called: {function_name}", flush=True)
                function_response = function_to_call(**function_args)
            except Exception as e:
                # Handle any errors during function execution
                function_response = f"Error executing {function_name}: {str(e)}"

        tool_output.append({"role": "tool","tool_call_id": tool_call.id,"content": json.dumps(function_response)})

    return tool_output     


In [ ]:
# Define the name of the person the chatbot is representing
# This is used to personalize the chatbot's responses
name = "Ed Donner"

# Read the LinkedIn PDF file using the pypdf library
reader = PdfReader("me/linkedin.pdf")
linkedin_text = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin_text += text

# Read the summary of the person the chatbot is representing
# This is used to provide context to the chatbot's responses
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [ ]:
# Define the system prompt for the chatbot which uses GPT-4o-mini model

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unanswered_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin_text}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
# 
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        # response_message = response.choices[0].message

        # messages.append(response_message)
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            response_message = response.choices[0].message
            tool_calls = response_message.tool_calls
            tool_outputs = handle_tool_calls(tool_calls)
            messages.append(response_message)
            messages.extend(tool_outputs)
        else:
            done = True
    return response_message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()